<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Github/capstone

/content/drive/MyDrive/Github/capstone


In [4]:
#!git pull   #check that git us up to date
#!git checkout -b shaptonstall # create a new branch

#!git push origin shaptonstall #commit changes to my branch

In [5]:
# to commit to github
#!git status # check untracked files
#!git add {filename} # add the files
#!git config --global user.email "sarahapt@umich.edu"
# !git commit

Import libraries

In [6]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta

In [7]:
!ls

 Capstone.ipynb		  README.md			       Webscraping.xlsx
 data_exploration.ipynb  'trivium_full_column_names (1).csv'


In [8]:
excel_file = 'Webscraping.xlsx'

# Load all sheets into a dictionary of DataFrames
webscraping = pd.read_excel('Webscraping.xlsx', sheet_name=None)



/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [9]:
webscraping.keys()
webscraping['Events and8']

,Last Scraped:,2023-07-21 00:00:00,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Date,Name,Category,Stage,Link,Format,Scraped
2,2023-07-16 00:00:00,Campionato Italiano di Categoria Breaking FIDS,1vs1 BBoy 12-15 C,32 @ Preselection,https://and8.dance/en/stats/reports/4459/1790/...,NaN,no
3,2023-07-16 00:00:00,Campionato Italiano di Categoria Breaking FIDS,1vs1 BBoy 12-15 C,Top 16 KnockOut,https://and8.dance/en/stats/reports/4459/1790/r/,Threefold,yes
4,2023-07-16 00:00:00,Campionato Italiano di Categoria Breaking FIDS,1vs1 BBoy 12-15 U,26 @ Preselection,https://and8.dance/en/stats/reports/4459/1791/...,NaN,no
...,...,...,...,...,...,...,...
1373,2014-12-28 00:00:00,Eve Dance 2014,1vs1 Popping,Top 8 Knock-Out,https://and8.dance/en/stats/reports/25/4/,NaN,no
1374,2014-12-28 00:00:00,Eve Dance 2014,2vs2 B-Boying,Top 8 Knock-Out,https://and8.dance/en/stats/reports/25/10/,NaN,no
1375,2014-11-08 00:00:00,Freestyle Session World Finals 2014,1vs1 Undisputed,Top 16 Knock-Out,https://and8.dance/en/stats/reports/84/298/r/,Round by Round (no 3rd place),no
1376,2014-10-17 00:00:00,BOTY International 1vs1 Battle 2014,1vs1 Undisputed,Top 16 Knock-Out,https://and8.dance/en/stats/reports/297/287/r/,Round by Round (no 3rd place),no


In [10]:
trivium=webscraping['Battles Trivium']
trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
trivium[['Event', 'Category']]= trivium[['Event', 'Category']].fillna(method='ffill')
trivium.head(2)

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,breaker 2 round wins,breaker 1 vote count,...,r5j8but2,r5j9over,r5j9tech,r5j9vari,r5j9perf,r5j9musi,r5j9crea,r5j9pers,r5j9but1,r5j9but2
0,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Ami,671,671,3,9,1,2,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Sunny,Anti,Anti,3,9,1,2,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
trivium

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,breaker 2 round wins,breaker 1 vote count,...,r5j8but2,r5j9over,r5j9tech,r5j9vari,r5j9perf,r5j9musi,r5j9crea,r5j9pers,r5j9but1,r5j9but2
0,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Ami,671,671,3,9,1,2,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Sunny,Anti,Anti,3,9,1,2,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Anti,671,671,3,9,0,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Ami,Sunny,Ami,3,9,3,0,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,671,Ayumi,671,2,9,2,0,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Stiv,Rerunn,Rerunn,2,5,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1522,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Piccolo,Rerunn,Rerunn,2,5,0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1523,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Piccolo,D-Matt,D-Matt,2,5,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1524,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Stiv,D-Matt,D-Matt,2,5,0,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
trivium_columns=list(trivium.columns)
exclude=['judge 1 name ', 'judge 2 name', 'judge 3 name', 'judge 4 name', 'judge 5 name', 'judge 6 name ', 'judge 7 name', 'judge 8 name', 'judge 9 name'  ]
trivium_columns= [x for x in trivium_columns if x not in exclude]
trivium=trivium[trivium_columns]

In [13]:
trivium['BattleID']= trivium.index
trivium

<ipython-input-13-445f84969105>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trivium['BattleID']= trivium.index


,Event,Category,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,breaker 2 round wins,breaker 1 vote count,...,r5j9over,r5j9tech,r5j9vari,r5j9perf,r5j9musi,r5j9crea,r5j9pers,r5j9but1,r5j9but2,BattleID
0,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Ami,671,671,3,9,1,2,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Sunny,Anti,Anti,3,9,1,2,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Anti,671,671,3,9,0,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,Ami,Sunny,Ami,3,9,3,0,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,671,Ayumi,671,2,9,2,0,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Stiv,Rerunn,Rerunn,2,5,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1521
1522,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Piccolo,Rerunn,Rerunn,2,5,0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1522
1523,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Piccolo,D-Matt,D-Matt,2,5,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1523
1524,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Stiv,D-Matt,D-Matt,2,5,0,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1524


In [14]:
category_sums_blue = {
    "Creativity": trivium.filter(like="crea").sum(axis=1),
    "Musicality": trivium.filter(like="musi").sum(axis=1),
    "Performance": trivium.filter(like="perf").sum(axis=1),
    "Personality": trivium.filter(like="pers").sum(axis=1),
    "Technique": trivium.filter(like="tech").sum(axis=1),
    "Variety": trivium.filter(like="vari").sum(axis=1),
    "Overall":  trivium.filter(like="over").sum(axis=1)
}

category_sums_red = {
    "Creativity": trivium.filter(like="crea").sum(axis=1)*-1,
    "Musicality": trivium.filter(like="musi").sum(axis=1)*-1,
    "Performance": trivium.filter(like="perf").sum(axis=1)*-1,
    "Personality": trivium.filter(like="pers").sum(axis=1)*-1,
    "Technique": trivium.filter(like="tech").sum(axis=1)*-1,
    "Variety": trivium.filter(like="vari").sum(axis=1)*-1,
    "Overall":  trivium.filter(like="over").sum(axis=1)*-1
}

In [15]:
scorecardBlue=pd.DataFrame({
    "Event": trivium['Event'],
    'Category': trivium['Category'],
    "BattleID": trivium["BattleID"],
    "Side": "BLUE",
    "Breaker": trivium["breaker 2 (blue)"],
        **category_sums_blue,
})

scorecardRed=pd.DataFrame({
    "Event": trivium['Event'],
    'Category': trivium['Category'],
    "BattleID": trivium["BattleID"],
    "Side": "RED",
    "Breaker": trivium["breaker 1 (red)"],
     **category_sums_red,
})

In [16]:
scorecard = pd.concat([scorecardBlue, scorecardRed], ignore_index=True)
scorecard[scorecard['BattleID']==0]

,Event,Category,BattleID,Side,Breaker,Creativity,Musicality,Performance,Personality,Technique,Variety,Overall
0,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,0,BLUE,671,-19.6,0.8,15.6,3.85,54.6,-7.07,48.18
1526,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,0,RED,Ami,19.6,-0.8,-15.6,-3.85,-54.6,7.07,-48.18


In [17]:
# scorecard[scorecard['Breaker']=="Ami"].mean(), scorecard[scorecard['Breaker']=="Ami"].count()

In [18]:
# scorecard[scorecard['Breaker']==671].mean(), scorecard[scorecard['Breaker']==671].count()

In [19]:
eventsAnd8=webscraping['Events and8'][1:].copy()
new_header= eventsAnd8.iloc[0]
eventsAnd8=eventsAnd8.drop(1)
eventsAnd8.columns = new_header
eventsAnd8.rename(columns={'Name':'Event'}, inplace=True)
eventsAnd8.drop_duplicates(subset=['Event'], inplace=True)

In [20]:
eventsAnd8.head(2)

1,Date,Event,Category,Stage,Link,Format,Scraped
2,2023-07-16 00:00:00,Campionato Italiano di Categoria Breaking FIDS,1vs1 BBoy 12-15 C,32 @ Preselection,https://and8.dance/en/stats/reports/4459/1790/...,NaN,no
26,2023-07-08 00:00:00,German Breaking Championship 2023,1vs1 B-Girls,21 @ Preselection,https://and8.dance/en/stats/reports/4437/1726/...,NaN,no


In [21]:
scorecard=pd.merge(scorecard, eventsAnd8[['Date', 'Event']], on='Event')


In [23]:
today = scorecard['Date'].max()
total_days = (today - scorecard['Date'].min()).days + 1  # Get the total number of days dynamically
scorecard['Weight'] = ((today - scorecard['Date']).dt.days / total_days) * 100
scorecard['Weight'] = 100 - scorecard['Weight']  # Reverse the weights so that newer dates have higher weights

In [25]:
scorecard[scorecard['BattleID']==0]

,Event,Category,BattleID,Side,Breaker,Creativity,Musicality,Performance,Personality,Technique,Variety,Overall,Date,Weight
0,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,0,BLUE,671,-19.6,0.8,15.6,3.85,54.6,-7.07,48.18,2023-02-24 00:00:00,100.0
128,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,0,RED,Ami,19.6,-0.8,-15.6,-3.85,-54.6,7.07,-48.18,2023-02-24 00:00:00,100.0
